##### Load Dependencies

In [32]:
import os
import numpy as np
import pandas as pd
from booggr import Error, ErrorDialog
from openai import OpenAI
from pydantic import BaseModel
import sklearn
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Perceptron
import ipywidgets as widgets, IPython, platform, ipywidgets, jupyterlab
from static import GptRequests, GptRoles, GptLanguages
from booggr import Error, ErrorDialog, ChatWindow, FileDialog, FileBrowser
import matplotlib
from matplotlib.colors import ListedColormap
from typing import Any, List, Tuple, Optional, Dict

##### Perceptron

In [5]:
class Perceptron( ):
	'''


		Purpose
		________

		Class to train models via fit function


		Parameters
		------------
		eta : float
		Learning rate (between 0.0 and 1.0)
		n_iter : int
		Passes over the training dataset.
		random_state : int
		Random num generator seed for random weight
		initialization.


		Attributes
		-----------
		w_ : 1d-array
		Weights after fitting.
		b_ : Scalar
		Bias unit after fitting.
		errors_ : list
		Number of misclassifications (updates) in each epoch.


	'''


	def __init__( self, eta=0.01, n_iter=50, random_state=42 ):
		"""

			Purpose
			_______
			Initializes Perceptron opbjects


			Parameters
			----------
			eta: flaot.
			The learning rate (between 0.0 and 1.0)

			n_iter: int
			Target values.

			random_state: int
			Epochs.

			Returns
			-------
			self : object

		"""
		self.eta = eta
		self.n_iter = n_iter
		self.random_state = random_state


	def fit( self, X, y ):
		"""

			Purpose
			_______
			Fit training values.


			Parameters
			----------
			X : {array-like}, shape = [n_examples, n_features]
			Training vec, where n_examples is the num of
			examples and n_features is the num of features.

			y : array-like, shape = [n_examples]
			Target values.

			Returns
			-------
			self : object

		"""
		try:
			if X is None:
				raise Exception( 'values is not provided.' )
			elif y is None:
				raise Exception( 'y is not provided.' )
			else:
				rgen = np.random.RandomState( self.random_state )
				self.w_ = rgen.normal( loc=0.0, scale=0.01, size=X.shape[ 1 ] )
				self.b_ = np.float64( 0. )
				self.errors_ = [ ]

				for _ in range( self.n_iter ):
					errors = 0

				for xi, target in zip( X, y ):
					update = self.eta * ( target - self.predict( xi ) )

				self.w_ += update * xi
				self.b_ += update
				errors += int( update != 0.0 )
				self.errors_.append( errors )
				return self
		except Exception as e:
			exception = Error( e )
			exception.module = 'Boo'
			exception.cause = 'Perceptron'
			exception.method = 'fit( self, X y )'
			error = ErrorDialog( exception )
			error.show( )


	def net_input( self, X ):
		"""

			Purpose
			_______
			Calculates net path

			Parameters
			----------
			X : {array-like}, shape = [n_examples, n_features]
			Training vec, where n_examples is the num of
			examples and n_features is the num of features.


			Returns
			-------
			np.array

		"""
		try:
			if X is None:
				raise Exception( 'Aurguent "X" is not provided.' )
			else:
				return np.dot( X, self.w_ ) + self.b_
		except Exception as e:
			exception = Error( e )
			exception.module = 'Boo'
			exception.cause = 'Perceptron'
			exception.method = 'net_input( self, X ):'
			error = ErrorDialog( exception )
			error.show( )


	def predict( self, X ):
		"""

			Purpose
			_______
			Calculates prediction

			Parameters
			----------
			X : {array-like}, shape = [n_examples, n_features]
			Training vec, where n_examples is the num of
			examples and n_features is the num of features.


			Returns
			-------
			np.array

		"""
		try:
			if X is None:
				raise Exception( 'Aurguent "X" is not provided.' )
			else:
				return np.where( self.net_input( X ) >= 0.0, 1, 0 )
		except Exception as e:
			exception = Error( e )
			exception.module = 'Boo'
			exception.cause = 'Perceptron'
			exception.method = 'predict( self, X )'
			error = ErrorDialog( exception )
			error.show( )


	def __dir__( self ) -> List[ str ]:
		'''
			Methods that returns a get_list of member names
			Returns: get_list[ str ]
		'''
		return [ 'fit', 'net_input', 'predict',
		         'w_', 'b_', 'errors_',
		         'n_iter', 'random_state', 'eta' ]

##### LinearGradientDescent

In [6]:
class LinearGradientDescent( ):
	"""

		Purpose
		___________
		Adaptive Linear Neuron classifier.

		Parameters
		------------
		eta : float
		Learning rate (between 0.0 and 1.0)
		n_iter : int
		Passes over the training dataset.
		random_state : int
		Random num generator seed for random weight initialization.

		Attributes
		-----------
		w_ : 1d-array
		Weights after fitting.
		b_ : Scalar
		Bias unit after fitting.
		losses_ : list
		Mean squared error loss function values in each epoch.

	"""


	def __init__( self, eta=0.01, n_iter=50, random_state=1 ):
		"""

			Purpose
			_______
			Initializes LinearGradientDescent opbjects


			Parameters
			----------
			eta: flaot=0.01
			The learning rate (between 0.0 and 1.0)

			n_iter: int: 50
			Target values.

			random_state: int:1
			Epochs.

			Returns
			-------
			self : object

		"""
		self.eta = eta
		self.n_iter = n_iter
		self.random_state = random_state


	def fit( self, X, y ):
		"""

			Fit training values.

			Parameters
			----------
			X : {array-like}, shape = [n_examples, n_features]
			Training vec, where n_examples
			is the num of examples and
			n_features is the num of features.

			y : array-like, shape = [n_examples]
			Target values.

			Returns
			-------
			self : object

		"""
		try:
			if X is None:
				raise Exception( 'Aurguent "X" is not provided.' )
			elif y is None:
				raise Exception( 'y is not provided.' )
			else:
				rgen = np.random.RandomState( self.random_state )
				self.w_ = rgen.normal( loc=0.0, scale=0.01,
					size=X.shape[ 1 ] )
				self.b_ = np.float_( 0. )
				self.losses_ = [ ]
				for i in range( self.n_iter ):
					net_input = self.net_input( X )

				output = self.activation( net_input )
				errors = (y - output)
				self.w_ += self.eta * 2.0 * X.T.dot( errors ) / X.shape[ 0 ]
				self.b_ += self.eta * 2.0 * errors.mean( )
				loss = (errors ** 2).mean( )
				self.losses_.append( loss )
				return self
		except Exception as e:
			exception = Error( e )
			exception.module = 'Boo'
			exception.cause = 'LinearGradientDescent'
			exception.method = 'fit( self, X, y )'
			error = ErrorDialog( exception )
			error.show( )


	def net_input( self, X ):
		"""

			Purpose
			_______
			Calculates net path

			Parameters
			----------
			X : {array-like}, shape = [n_examples, n_features]
			Training vec, where n_examples is the num of
			examples and n_features is the num of features.


			Returns
			-------
			np.array

		"""
		try:
			if X is None:
				raise Exception( 'values is not provided.' )
			else:
				return np.dot( X, self.w_ ) + self.b_
		except Exception as e:
			exception = Error( e )
			exception.module = 'Boo'
			exception.cause = 'LinearGradientDescent'
			exception.method = 'net_input( self, X )'
			error = ErrorDialog( exception )
			error.show( )


	def activation( self, X ):
		"""

			Purpose
			_______
			Computes linear activation

			Parameters
			----------
			X : {array-like}, shape = [n_examples, n_features]
			Training vec, where n_examples is the num of
			examples and n_features is the num of features.


			Returns
			-------
			X : {array-like}, shape = [n_examples, n_features]
			Training vec, where n_examples is the num of
			examples and n_features is the num of features.

		"""
		try:
			if X is None:
				raise Exception( 'Aurguent "X" is not provided.' )
			else:
				return X
		except Exception as e:
			exception = Error( e )
			exception.module = 'Boo'
			exception.cause = 'LinearGradientDescent'
			exception.method = 'activation( self, X )'
			error = ErrorDialog( exception )
			error.show( )


	def predict( self, X ):
		"""

			Purpose
			_______
			Computes linear activation

			Parameters
			----------
			X : {array-like}, shape = [n_examples, n_features]
			Training vec, where n_examples is the num of
			examples and n_features is the num of features.


			Returns
			-------
			np.array

		"""
		try:
			if X is None:
				raise Exception( 'Aurguent "X" is not provided.' )
			else:
				return np.where( self.activation( self.net_input( X ) ) >= 0.5, 1, 0 )
		except Exception as e:
			exception = Error( e )
			exception.module = 'Boo'
			exception.cause = 'LinearGradientDescent'
			exception.method = 'predict( self, values )'
			error = ErrorDialog( exception )
			error.show( )


	def __dir__( self ) -> list[ str ]:
		'''

			Methods that returns a get_list of member names
			Returns: get_list[ str ]

		'''
		return [ 'fit', 'net_input', 'activation',
		         'predict', 'losses_', 'b_', 'w_',
		         'n_iter', 'eta', 'random_state'  ]

##### StochasticGradientDescent

In [7]:
class AdalineSGD:
	"""

		Purpose:
		____________
		Adaptive Linear Neuron Classifier.

		Parameters
		------------
		eta : float
		Learning rate (between 0.0 and 1.0)
		n_iter : int
		Passes over the training dataset.
		shuffle : bool (default: True)
		Shuffles training data every epoch if True to prevent
		cycles.
		random_state : int
		Random number generator seed for random weight
		initialization.

		Attributes
		-----------
		w_ : 1d-array
		Weights after fitting.

		b_ : Scalar
		Bias unit after fitting.

		losses_ : list
		Mean squared error loss function value averaged over all
		training examples in each epoch.

	"""
	def __init__( self, eta=0.01, n_iter=10, shuffle=True, random_state=None ):
		self.eta = eta
		self.n_iter = n_iter
		self.w_initialized = False
		self.shuffle = shuffle
		self.random_state = random_state


	def fit(self, X, y):
		"""

			Purpose:
			__________
			Fit training data.

			Parameters
			----------
			X : {array-like}, shape = [n_examples, n_features]
			Training vectors, where n_examples is the number of
			examples and n_features is the number of features.
			y : array-like, shape = [n_examples]
			Target values.

			Returns
			-------
			self : object

		"""
		self._initialize_weights(X.shape[1])
		self.losses_ = []
		for i in range(self.n_iter):
			if self.shuffle:
				X, y = self._shuffle(X, y)
		losses = []
		for xi, target in zip(X, y):
			losses.append(self._update_weights(xi, target))
			avg_loss = np.mean(losses)
			self.losses_.append(avg_loss)
			return self


	def partial_fit( self, X, y ):
		"""

			Fit training data
			without reinitializing the weights

		"""
		if not self.w_initialized:
			self._initialize_weights( X.shape[ 1 ] )
			if y.ravel( ).shape[ 0 ] > 1:
				for xi, target in zip( X, y ):
					self._update_weights( xi, target )
			else:
				self._update_weights( X,  y)
				return self


	def _shuffle(self, X, y):
		"""

			Shuffle training data

		"""
		r = self.rgen.permutation( len( y ) )
		return X[ r ], y[ r ]


	def _initialize_weights( self, m ):
		"""

			Initialize weights to
			small random numbers

		"""
		self.rgen = np.random.RandomState( self.random_state )
		self.w_ = self.rgen.normal( loc=0.0, scale=0.01, size=m )
		self.b_ = np.float_( 0. )
		self.w_initialized = True


	def _update_weights( self, xi, target ):
		"""

			Apply Adaline learning
			rule to update the weights

		"""
		output = self.activation( self.net_input( xi ) )
		error = ( target - output )
		self.w_ += self.eta * 2.0 * xi * ( error )
		self.b_ += self.eta * 2.0 * error
		loss = error**2
		return loss

##### Decision Region Plot

In [8]:
def plot_decision_regions( X, y, classifier, resolution=0.02 ):
	# setup marker generator and color map
	markers = ( 'o', 's', '^', 'v', '<' )
	colors = ( 'red', 'blue', 'lightgreen', 'gray', 'cyan' )
	cmap = ListedColormap( colors[ :len( np.unique( y ) ) ] )
	# plot the decision surface
	x1_min, x1_max = X[ :, 0 ].min( ) - 1, X[ :, 0 ].max( ) + 1
	x2_min, x2_max = X[ :, 1 ].min( ) - 1, X[ :, 1 ].max( ) + 1
	xx1, xx2 = np.meshgrid( np.arange( x1_min, x1_max, resolution ),
	np.arange( x2_min, x2_max, resolution ) )
	lab = classifier.predict( np.array( [ xx1.ravel( ), xx2.ravel( ) ] ).T )
	lab = lab.reshape( xx1.shape )
	plt.contourf( xx1, xx2, lab, alpha=0.3, cmap=cmap )
	plt.xlim( xx1.min( ), xx1.max( ) )
	plt.ylim( xx2.min( ), xx2.max( ) )
	# plot class examples
	for idx, cl in enumerate( np.unique( y ) ):
		plt.scatter( x=X[ y == cl, 0 ], y=X[ y == cl, 1 ], alpha=0.8,
		c=colors[ idx ], marker=markers[ idx ], label=f'Class {cl}', edgecolor='black' )

##### Balance Data

In [9]:
# Load the Excel file
file_path_balances = r'C:\Users\terry\source\repos\Boo\data\excel\Account Balances.xlsx'
df_balances = pd.read_excel( file_path_balances, sheet_name='Data' )
numeric_columns = [ 'CarryoverAuthority', 'CarryoverAdjustments', 'AnnualAppropriations',
                    'BorrowingAuthority', 'ContractAuthority', 'OffsettingReceipts',
                    'Obligations', 'Recoveries', 'UnobligatedBalance', 'Outlays', 'TotalResources' ]
numeric_subset = [ 'AnnualAppropriations', 'CarryoverAuthority', 'UnobligatedBalance', 'Obligations', 'Outlays' ]

# Filter values
all = [ 'AccountBalancesId', 'AgencyIdentifier', 'AgencyName', 'BeginningPeriodOfAvailability',
        'EndingPeriodOfAvailability', 'Availability', 'MainAccountCode', 'SubAccountCode',
        'TreasuryAccountSymbol', 'TreasuryAccountName', 'BudgetFunction', 'BudgetSubFunction',
        'FederalAccountSymbol', 'FederalAccountName', 'LastModified',
        'SubmissionPeriod' ] + numeric_columns
fields = [ 'AgencyIdentifier', 'AgencyName', 'TreasuryAccountSymbol', 'TreasuryAccountName', 'BudgetFunction' ]
subset = fields + numeric_columns

# Define datasets
df_dataset = df_balances[ all ].fillna( 0 )
df_subset = df_balances[ subset ].fillna( 0 )
df_numeric = df_balances[ numeric_columns ].fillna( 0 )

##### Iris Data

In [ ]:
iris = datasets.load_iris( )
X = iris.data[ :, [ 2, 3 ] ]
y = iris.target
np.unique( y )
np.bincount( y )

In [ ]:
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.3, random_state=1, stratify=y )
np.bincount( y_train )
np.bincount( y_test )

In [30]:
sc = StandardScaler( )
sc.fit( X_train )
X_train_std = sc.transform( X_train )
X_test_std = sc.transform( X_test )
len( X_test_std )

45

In [ ]:
ppn = Perceptron( eta0=0.1, random_state=1 )
ppn.fit( X_train_std, y_train )

In [34]:
y_pred = ppn.predict( X_test_std )
print( 'Misclassified examples: %d' % ( y_test != y_pred ).sum( ) )

Misclassified examples: 1
